<a href="https://colab.research.google.com/github/swethag04/llm-projects/blob/main/llamaindex_tutorial/llama_index_rag_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# wrap around the text on screen
import textwrap

In [ ]:
# !pip install openai
# !pip install tiktoken
# !pip install cohere
# !pip install pypdf
# !pip install sentence-transformers
# !pip install -q "huggingface_hub[inference]"
# !pip install -U llama-index
# !pip install langchain

In [4]:
import os
import openai
#from google.colab import userdata
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
#os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HUGGINGFACE_API_KEY')
#openai.api_key = os.getenv("OPENAI_API_KEY")

In [18]:
import pandas as pd
import regex as re
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    OpenAIEmbedding,
    load_index_from_storage,
    get_response_synthesizer
)
from llama_index.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import nest_asyncio
nest_asyncio.apply()

In [6]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine, CitationQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor


In [7]:
# Load the document
reader = SimpleDirectoryReader(input_dir = '/content/sample_data')
emp_doc = reader.load_data()
print(f"Loaded{len(emp_doc)} docs")

Loaded150 docs


In [8]:
print(emp_doc[1])

Doc ID: ad18290e-d80e-451f-867c-09228534a64f
Text: ●Belittling conduct : name calling; playing “pranks” on a
person; making fun of someone or telling jokes at their expense;
taking, vandal izing, or otherwise damaging a person’s personal or
work property; and spreading false information or rumors about
someone; seeking submission or misuse of power, authority, rank,
status, or other privilege ●M...


In [9]:
## Customized  prompt to
from llama_index.prompts import PromptTemplate
template = (
    "You are a search service agent helping answer questions regarding employee policy for new and existing employees"
    "You are provided with employees workplace policy document"
    "Please provide an answer based solely on the provided sources. "
    "Show page number from the source document "
    "Every answer should include only one source citation. "
    "Show only one source that was referenced in the answer"
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

In [10]:
# Create an index from the document
# increased chunk size and chunk overlap for better response
index = VectorStoreIndex.from_documents(emp_doc, service_context=ServiceContext.from_defaults(chunk_size=1000, chunk_overlap=200))


In [11]:
index.storage_context.persist(persist_dir = '/content/sample_data')


In [12]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5, ## default value = 2
)

In [13]:
## Query engine for without citation
# query_engine = RetrieverQueryEngine.from_args(
#     retriever=retriever,
#    # service_context = service_context,
#     node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
#     response_mode = "compact",
#     text_qa_template=qa_template
# )


In [14]:
# query engine with citation
citation_engine = CitationQueryEngine.from_args(
    index,
    citation_chunk_size=1000,
    citation_chunk_overlap = 200,
    citation_qa_template=qa_template,
     retriever=retriever,
    response_mode = "compact",
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)]
)


In [15]:
citation_engine.get_prompts()

{'response_synthesizer:text_qa_template': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are a search service agent helping answer questions regarding employee policy for new and existing employeesYou are provided with employees workplace policy documentPlease provide an answer based solely on the provided sources. Show page number from the source document Every answer should include only one source citation. Show only one source that was referenced in the answerOnly cite a source when you are explicitly referencing it. If none of the sources are helpful, you should indicate that. ---------------------\n{context_str}\n---------------------\nGiven this information, please answer the question: {query_str}\n'),
 'response_synthesizer:refine_template': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, tem

In [20]:
ques_list = [
    "What is the severance pay for an employee during a layoff?",
    "What is the pay for night duty?",
    "How much life insurance is provided?",
]
for q in ques_list:
  print(f"Question: {q} ")
  ans = citation_engine.query(q)
  print(textwrap.fill(f"Answer: {ans}", 80))
  sources=re.findall(r"(Source \d*)", ans.response)
  for s in sources:
    index = int(s.split()[1])
    print(ans.source_nodes[index-1].node.get_content())
  print("--------------------------------------------")
  print("--------------------------------------------")


Question: What is the severance pay for an employee during a layoff? 
Answer: The severance pay for an employee during a layoff is one week of
severance pay for each full year of continuous service in any Category 18
appointment, up to a maximum of nine weeks of severance pay. This severance pay
is calculated at the employee's base hourly rate. (Source: Page 27)
--------------------------------------------
--------------------------------------------
Question: What is the pay for night duty? 
Answer: Employees shall be paid eight percent (8%) more than the base rate for
each hour worked between 5:00 p.m. and 7:00 a.m., provided that the employees'
regular shift includes at least one (1) hour between 5:00 p.m. and 7:00 a.m.
(Source 1, page 46)
Source 1:
ARTICLE III – PAY, HOURS AND BENEFITS  
 
MEMORANDUM OF UNDERSTANDING, FY 2022 - 2024 
CITY AND COUNTY OF SAN FRANCISCO AND  
IFPTE, LOCAL 21  
 
40 19. Standby Pay  
256. Employees who receive this premium shall respond immediately when

In [ ]:
# Pretty response
# from llama_index.response.notebook_utils import (
#     display_source_node,
#     display_response,)
# display_response(
#     ans, source_length=1000, show_source=True, show_source_metadata=True
# )
